In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr
from scipy.stats import anderson, shapiro,kstest
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics 

In [2]:
mydata = pd.read_csv('h1n1_vaccine_prediction.csv')
mydata.head()

,unique_id,h1n1_worry,h1n1_awareness,antiviral_medication,contact_avoidance,bought_face_mask,wash_hands_frequently,avoid_large_gatherings,reduced_outside_home_cont,avoid_touch_face,...,race,sex,income_level,marital_status,housing_status,employment,census_msa,no_of_adults,no_of_children,h1n1_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,Non-MSA,0.0,0.0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,White,Male,Below Poverty,Not Married,Rent,Employed,"MSA, Not Principle City",0.0,0.0,0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,"MSA, Not Principle City",2.0,0.0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,"MSA, Principle City",0.0,0.0,0
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,"MSA, Not Principle City",1.0,0.0,0


In [3]:
mydata.shape

(26707, 34)

In [4]:
mydata.columns

Index(['unique_id', 'h1n1_worry', 'h1n1_awareness', 'antiviral_medication',
       'contact_avoidance', 'bought_face_mask', 'wash_hands_frequently',
       'avoid_large_gatherings', 'reduced_outside_home_cont',
       'avoid_touch_face', 'dr_recc_h1n1_vacc', 'dr_recc_seasonal_vacc',
       'chronic_medic_condition', 'cont_child_undr_6_mnths',
       'is_health_worker', 'has_health_insur', 'is_h1n1_vacc_effective',
       'is_h1n1_risky', 'sick_from_h1n1_vacc', 'is_seas_vacc_effective',
       'is_seas_risky', 'sick_from_seas_vacc', 'age_bracket', 'qualification',
       'race', 'sex', 'income_level', 'marital_status', 'housing_status',
       'employment', 'census_msa', 'no_of_adults', 'no_of_children',
       'h1n1_vaccine'],
      dtype='object')

In [5]:
mydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   unique_id                  26707 non-null  int64  
 1   h1n1_worry                 26615 non-null  float64
 2   h1n1_awareness             26591 non-null  float64
 3   antiviral_medication       26636 non-null  float64
 4   contact_avoidance          26499 non-null  float64
 5   bought_face_mask           26688 non-null  float64
 6   wash_hands_frequently      26665 non-null  float64
 7   avoid_large_gatherings     26620 non-null  float64
 8   reduced_outside_home_cont  26625 non-null  float64
 9   avoid_touch_face           26579 non-null  float64
 10  dr_recc_h1n1_vacc          24547 non-null  float64
 11  dr_recc_seasonal_vacc      24547 non-null  float64
 12  chronic_medic_condition    25736 non-null  float64
 13  cont_child_undr_6_mnths    25887 non-null  flo

### EDA

In [6]:
mydata.describe()

,unique_id,h1n1_worry,h1n1_awareness,antiviral_medication,contact_avoidance,bought_face_mask,wash_hands_frequently,avoid_large_gatherings,reduced_outside_home_cont,avoid_touch_face,...,has_health_insur,is_h1n1_vacc_effective,is_h1n1_risky,sick_from_h1n1_vacc,is_seas_vacc_effective,is_seas_risky,sick_from_seas_vacc,no_of_adults,no_of_children,h1n1_vaccine
count,26707.000000,26615.000000,26591.000000,26636.000000,26499.000000,26688.000000,26665.000000,26620.00000,26625.000000,26579.000000,...,14433.00000,26316.000000,26319.000000,26312.000000,26245.000000,26193.000000,26170.000000,26458.000000,26458.000000,26707.000000
mean,13353.000000,1.618486,1.262532,0.048844,0.725612,0.068982,0.825614,0.35864,0.337315,0.677264,...,0.87972,3.850623,2.342566,2.357670,4.025986,2.719162,2.118112,0.886499,0.534583,0.212454
std,7709.791156,0.910311,0.618149,0.215545,0.446214,0.253429,0.379448,0.47961,0.472802,0.467531,...,0.32530,1.007436,1.285539,1.362766,1.086565,1.385055,1.332950,0.753422,0.928173,0.409052
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,6676.500000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,...,1.00000,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000,0.000000
50%,13353.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,1.000000,...,1.00000,4.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000,0.000000
75%,20029.500000,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,...,1.00000,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000,0.000000
max,26706.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000,1.000000


### Checking for missing values

In [7]:
mydata.isnull().sum()

unique_id                        0
h1n1_worry                      92
h1n1_awareness                 116
antiviral_medication            71
contact_avoidance              208
bought_face_mask                19
wash_hands_frequently           42
avoid_large_gatherings          87
reduced_outside_home_cont       82
avoid_touch_face               128
dr_recc_h1n1_vacc             2160
dr_recc_seasonal_vacc         2160
chronic_medic_condition        971
cont_child_undr_6_mnths        820
is_health_worker               804
has_health_insur             12274
is_h1n1_vacc_effective         391
is_h1n1_risky                  388
sick_from_h1n1_vacc            395
is_seas_vacc_effective         462
is_seas_risky                  514
sick_from_seas_vacc            537
age_bracket                      0
qualification                 1407
race                             0
sex                              0
income_level                  4423
marital_status                1408
housing_status      

### Splitting Categorical Datas

In [8]:
cat = mydata.select_dtypes(include="object")
cat

,age_bracket,qualification,race,sex,income_level,marital_status,housing_status,employment,census_msa
0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,Non-MSA
1,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,"MSA, Not Principle City"
2,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,"MSA, Not Principle City"
3,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,"MSA, Principle City"
4,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,"MSA, Not Principle City"
...,...,...,...,...,...,...,...,...,...
26702,65+ Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,Non-MSA
26703,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Rent,Employed,"MSA, Principle City"
26704,55 - 64 Years,Some College,White,Female,NaN,Not Married,Own,NaN,"MSA, Not Principle City"
26705,18 - 34 Years,Some College,Hispanic,Female,"<= $75,000, Above Poverty",Married,Rent,Employed,Non-MSA


### Splitting Numerical Datas

In [9]:
num = mydata.select_dtypes(exclude="object")
num

,unique_id,h1n1_worry,h1n1_awareness,antiviral_medication,contact_avoidance,bought_face_mask,wash_hands_frequently,avoid_large_gatherings,reduced_outside_home_cont,avoid_touch_face,...,has_health_insur,is_h1n1_vacc_effective,is_h1n1_risky,sick_from_h1n1_vacc,is_seas_vacc_effective,is_seas_risky,sick_from_seas_vacc,no_of_adults,no_of_children,h1n1_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,3.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,5.0,4.0,4.0,4.0,2.0,4.0,0.0,0.0,0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,3.0,1.0,1.0,4.0,1.0,2.0,2.0,0.0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,NaN,3.0,3.0,5.0,5.0,4.0,1.0,0.0,0.0,0
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,NaN,3.0,3.0,2.0,3.0,1.0,4.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,3.0,1.0,1.0,5.0,2.0,2.0,0.0,0.0,0
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,4.0,2.0,2.0,5.0,1.0,1.0,1.0,0.0,0
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,4.0,4.0,2.0,5.0,4.0,2.0,0.0,0.0,0
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,3.0,1.0,2.0,2.0,1.0,2.0,1.0,0.0,0


### Checking Missing Values in Categorical Data

In [10]:
cat.isnull().sum()

age_bracket          0
qualification     1407
race                 0
sex                  0
income_level      4423
marital_status    1408
housing_status    2042
employment        1463
census_msa           0
dtype: int64

In [11]:
cat.isnull().sum()/cat.shape[0]*100

age_bracket        0.000000
qualification      5.268282
race               0.000000
sex                0.000000
income_level      16.561201
marital_status     5.272026
housing_status     7.645936
employment         5.477965
census_msa         0.000000
dtype: float64

### Replacing the missing values using mode function

In [12]:
cat["income_level"].value_counts(normalize=True)

<= $75,000, Above Poverty    0.573371
> $75,000                    0.305600
Below Poverty                0.121029
Name: income_level, dtype: float64

In [13]:
cat["income_level"].replace({"Below Poverty" : 0,"<= $75,000, Above Poverty":1,"> $75,000":2},inplace=True)
cat["income_level"]

0        0.0
1        0.0
2        1.0
3        0.0
4        1.0
        ... 
26702    1.0
26703    1.0
26704    NaN
26705    1.0
26706    1.0
Name: income_level, Length: 26707, dtype: float64

In [14]:
cat["income_level"].mode()

0    1.0
Name: income_level, dtype: float64

In [15]:
cat['income_level'].fillna(1.0,inplace=True)
cat['income_level']

0        0.0
1        0.0
2        1.0
3        0.0
4        1.0
        ... 
26702    1.0
26703    1.0
26704    1.0
26705    1.0
26706    1.0
Name: income_level, Length: 26707, dtype: float64

In [16]:
cat['qualification'].value_counts(normalize=True)

College Graduate    0.399091
Some College        0.278379
12 Years            0.229130
< 12 Years          0.093399
Name: qualification, dtype: float64

In [17]:
cat["qualification"].replace({"< 12 Years" : 0,"12 Years":1,"Some College":2,"College Graduate":3},inplace=True)
cat["qualification"]

0        0.0
1        1.0
2        3.0
3        1.0
4        2.0
        ... 
26702    2.0
26703    3.0
26704    2.0
26705    2.0
26706    2.0
Name: qualification, Length: 26707, dtype: float64

In [18]:
cat["qualification"].mode()

0    3.0
Name: qualification, dtype: float64

In [19]:
cat['qualification'].fillna(3.0,inplace=True)
cat['qualification']

0        0.0
1        1.0
2        3.0
3        1.0
4        2.0
        ... 
26702    2.0
26703    3.0
26704    2.0
26705    2.0
26706    2.0
Name: qualification, Length: 26707, dtype: float64

In [20]:
cat.isnull().sum()

age_bracket          0
qualification        0
race                 0
sex                  0
income_level         0
marital_status    1408
housing_status    2042
employment        1463
census_msa           0
dtype: int64

In [21]:
cat['marital_status'].value_counts(normalize=True)

Married        0.535792
Not Married    0.464208
Name: marital_status, dtype: float64

In [22]:
cat['marital_status'].replace({"Not Married":0,"Married":1},inplace=True)
cat['marital_status']

0        0.0
1        0.0
2        0.0
3        0.0
4        1.0
        ... 
26702    0.0
26703    0.0
26704    0.0
26705    1.0
26706    1.0
Name: marital_status, Length: 26707, dtype: float64

In [23]:
cat['marital_status'].mode()

0    1.0
Name: marital_status, dtype: float64

In [24]:
cat['marital_status'].fillna('1',inplace=True)
cat['marital_status']

0        0.0
1        0.0
2        0.0
3        0.0
4        1.0
        ... 
26702    0.0
26703    0.0
26704    0.0
26705    1.0
26706    1.0
Name: marital_status, Length: 26707, dtype: object

In [25]:
cat['housing_status'].value_counts(normalize=True)

Own     0.759619
Rent    0.240381
Name: housing_status, dtype: float64

In [26]:
cat['housing_status'].replace({"Rent":0,"Own":1},inplace=True)
cat['housing_status']

0        1.0
1        0.0
2        1.0
3        0.0
4        1.0
        ... 
26702    1.0
26703    0.0
26704    1.0
26705    0.0
26706    1.0
Name: housing_status, Length: 26707, dtype: float64

In [27]:
cat['housing_status'].mode()

0    1.0
Name: housing_status, dtype: float64

In [28]:
cat['housing_status'].fillna(1.0,inplace=True)
cat['housing_status']

0        1.0
1        0.0
2        1.0
3        0.0
4        1.0
        ... 
26702    1.0
26703    0.0
26704    1.0
26705    0.0
26706    1.0
Name: housing_status, Length: 26707, dtype: float64

### Handling Missing Values using user-defined function for categorical data

In [29]:
for i in cat.columns:
    cat[i] = cat[i].fillna(cat[i].dropna().mode().values[0])

In [30]:
cat.isnull().sum()

age_bracket       0
qualification     0
race              0
sex               0
income_level      0
marital_status    0
housing_status    0
employment        0
census_msa        0
dtype: int64

In [31]:
cat.isnull().sum()/cat.shape[0]

age_bracket       0.0
qualification     0.0
race              0.0
sex               0.0
income_level      0.0
marital_status    0.0
housing_status    0.0
employment        0.0
census_msa        0.0
dtype: float64

In [32]:
num = mydata.select_dtypes(exclude="object")
num

,unique_id,h1n1_worry,h1n1_awareness,antiviral_medication,contact_avoidance,bought_face_mask,wash_hands_frequently,avoid_large_gatherings,reduced_outside_home_cont,avoid_touch_face,...,has_health_insur,is_h1n1_vacc_effective,is_h1n1_risky,sick_from_h1n1_vacc,is_seas_vacc_effective,is_seas_risky,sick_from_seas_vacc,no_of_adults,no_of_children,h1n1_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,3.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,5.0,4.0,4.0,4.0,2.0,4.0,0.0,0.0,0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,3.0,1.0,1.0,4.0,1.0,2.0,2.0,0.0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,NaN,3.0,3.0,5.0,5.0,4.0,1.0,0.0,0.0,0
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,NaN,3.0,3.0,2.0,3.0,1.0,4.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,NaN,3.0,1.0,1.0,5.0,2.0,2.0,0.0,0.0,0
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,4.0,2.0,2.0,5.0,1.0,1.0,1.0,0.0,0
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,4.0,4.0,2.0,5.0,4.0,2.0,0.0,0.0,0
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,3.0,1.0,2.0,2.0,1.0,2.0,1.0,0.0,0


### Handling Missing Values using user-defined function in numerical data

In [33]:
for i in num.columns:
    num[i] = num[i].fillna(num[i].dropna().mode().values[0])

In [34]:
num.isnull().sum()

unique_id                    0
h1n1_worry                   0
h1n1_awareness               0
antiviral_medication         0
contact_avoidance            0
bought_face_mask             0
wash_hands_frequently        0
avoid_large_gatherings       0
reduced_outside_home_cont    0
avoid_touch_face             0
dr_recc_h1n1_vacc            0
dr_recc_seasonal_vacc        0
chronic_medic_condition      0
cont_child_undr_6_mnths      0
is_health_worker             0
has_health_insur             0
is_h1n1_vacc_effective       0
is_h1n1_risky                0
sick_from_h1n1_vacc          0
is_seas_vacc_effective       0
is_seas_risky                0
sick_from_seas_vacc          0
no_of_adults                 0
no_of_children               0
h1n1_vaccine                 0
dtype: int64

### One Hot Encoder

In [35]:
cat1 = pd.get_dummies(cat,drop_first=True)
cat1

,qualification,income_level,housing_status,age_bracket_35 - 44 Years,age_bracket_45 - 54 Years,age_bracket_55 - 64 Years,age_bracket_65+ Years,race_Hispanic,race_Other or Multiple,race_White,sex_Male,marital_status_1.0,marital_status_1,employment_Not in Labor Force,employment_Unemployed,"census_msa_MSA, Principle City",census_msa_Non-MSA
0,0.0,0.0,1.0,0,0,1,0,0,0,1,0,0,0,1,0,0,1
1,1.0,0.0,0.0,1,0,0,0,0,0,1,1,0,0,0,0,0,0
2,3.0,1.0,1.0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
3,1.0,0.0,0.0,0,0,0,1,0,0,1,0,0,0,1,0,1,0
4,2.0,1.0,1.0,0,1,0,0,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,1.0,1.0,0,0,0,1,0,0,1,0,0,0,1,0,0,1
26703,3.0,1.0,0.0,0,0,0,0,0,0,1,1,0,0,0,0,1,0
26704,2.0,1.0,1.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
26705,2.0,1.0,0.0,0,0,0,0,1,0,0,0,1,0,0,0,0,1


In [36]:
ml_data = pd.concat([cat1,num],axis=1)

In [37]:
ml_data.isna().sum()

qualification                     0
income_level                      0
housing_status                    0
age_bracket_35 - 44 Years         0
age_bracket_45 - 54 Years         0
age_bracket_55 - 64 Years         0
age_bracket_65+ Years             0
race_Hispanic                     0
race_Other or Multiple            0
race_White                        0
sex_Male                          0
marital_status_1.0                0
marital_status_1                  0
employment_Not in Labor Force     0
employment_Unemployed             0
census_msa_MSA, Principle City    0
census_msa_Non-MSA                0
unique_id                         0
h1n1_worry                        0
h1n1_awareness                    0
antiviral_medication              0
contact_avoidance                 0
bought_face_mask                  0
wash_hands_frequently             0
avoid_large_gatherings            0
reduced_outside_home_cont         0
avoid_touch_face                  0
dr_recc_h1n1_vacc           

## MODEL BUILDING

### Logistic Algorithm

In [38]:
y = ml_data["h1n1_vaccine"]
x = ml_data.drop("h1n1_vaccine",axis=1)

In [39]:
x

,qualification,income_level,housing_status,age_bracket_35 - 44 Years,age_bracket_45 - 54 Years,age_bracket_55 - 64 Years,age_bracket_65+ Years,race_Hispanic,race_Other or Multiple,race_White,...,is_health_worker,has_health_insur,is_h1n1_vacc_effective,is_h1n1_risky,sick_from_h1n1_vacc,is_seas_vacc_effective,is_seas_risky,sick_from_seas_vacc,no_of_adults,no_of_children
0,0.0,0.0,1.0,0,0,1,0,0,0,1,...,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0
1,1.0,0.0,0.0,1,0,0,0,0,0,1,...,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,0.0,0.0
2,3.0,1.0,1.0,0,0,0,0,0,0,1,...,0.0,1.0,3.0,1.0,1.0,4.0,1.0,2.0,2.0,0.0
3,1.0,0.0,0.0,0,0,0,1,0,0,1,...,0.0,1.0,3.0,3.0,5.0,5.0,4.0,1.0,0.0,0.0
4,2.0,1.0,1.0,0,1,0,0,0,0,1,...,0.0,1.0,3.0,3.0,2.0,3.0,1.0,4.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,1.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,3.0,1.0,1.0,5.0,2.0,2.0,0.0,0.0
26703,3.0,1.0,0.0,0,0,0,0,0,0,1,...,1.0,1.0,4.0,2.0,2.0,5.0,1.0,1.0,1.0,0.0
26704,2.0,1.0,1.0,0,0,1,0,0,0,1,...,0.0,1.0,4.0,4.0,2.0,5.0,4.0,2.0,0.0,0.0
26705,2.0,1.0,0.0,0,0,0,0,1,0,0,...,0.0,0.0,3.0,1.0,2.0,2.0,1.0,2.0,1.0,0.0


In [40]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=16)

In [42]:
xtrain

,qualification,income_level,housing_status,age_bracket_35 - 44 Years,age_bracket_45 - 54 Years,age_bracket_55 - 64 Years,age_bracket_65+ Years,race_Hispanic,race_Other or Multiple,race_White,...,is_health_worker,has_health_insur,is_h1n1_vacc_effective,is_h1n1_risky,sick_from_h1n1_vacc,is_seas_vacc_effective,is_seas_risky,sick_from_seas_vacc,no_of_adults,no_of_children
15255,2.0,0.0,1.0,0,0,0,0,1,0,0,...,0.0,1.0,3.0,2.0,4.0,4.0,4.0,2.0,1.0,1.0
24029,3.0,1.0,1.0,0,1,0,0,0,0,1,...,0.0,1.0,3.0,1.0,4.0,4.0,2.0,2.0,2.0,0.0
5882,3.0,2.0,0.0,0,0,0,0,0,0,1,...,0.0,1.0,5.0,2.0,2.0,5.0,2.0,2.0,1.0,0.0
26065,3.0,1.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,4.0,2.0,5.0,4.0,2.0,1.0,1.0
6106,3.0,1.0,1.0,1,0,0,0,0,0,1,...,0.0,1.0,4.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17729,2.0,1.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,2.0,1.0,5.0,4.0,1.0,0.0,0.0
581,3.0,1.0,0.0,0,0,0,0,0,0,1,...,0.0,1.0,4.0,4.0,4.0,5.0,4.0,2.0,0.0,0.0
2169,3.0,2.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,1.0,1.0,5.0,2.0,1.0,1.0,0.0
15598,3.0,1.0,1.0,0,1,0,0,0,0,1,...,0.0,1.0,3.0,2.0,4.0,3.0,2.0,2.0,2.0,2.0


In [43]:
model = LogisticRegression(max_iter=5000)
model.fit(xtrain,ytrain)

LogisticRegression(max_iter=5000)

In [44]:
pred = model.predict(xtest)

In [46]:
print(classification_report(pred,ytest))
print(confusion_matrix(pred,ytest))

              precision    recall  f1-score   support

           0       0.94      0.86      0.90      6945
           1       0.41      0.64      0.50      1068

    accuracy                           0.83      8013
   macro avg       0.67      0.75      0.70      8013
weighted avg       0.87      0.83      0.84      8013

[[5951  994]
 [ 385  683]]


In [47]:
y.value_counts(normalize=True)

0    0.787546
1    0.212454
Name: h1n1_vaccine, dtype: float64

### Decision Tree Algorithm

In [49]:
y = ml_data["h1n1_vaccine"]
x = ml_data.drop("h1n1_vaccine",axis=1)

In [51]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=16)

In [52]:
xtrain

,qualification,income_level,housing_status,age_bracket_35 - 44 Years,age_bracket_45 - 54 Years,age_bracket_55 - 64 Years,age_bracket_65+ Years,race_Hispanic,race_Other or Multiple,race_White,...,is_health_worker,has_health_insur,is_h1n1_vacc_effective,is_h1n1_risky,sick_from_h1n1_vacc,is_seas_vacc_effective,is_seas_risky,sick_from_seas_vacc,no_of_adults,no_of_children
15255,2.0,0.0,1.0,0,0,0,0,1,0,0,...,0.0,1.0,3.0,2.0,4.0,4.0,4.0,2.0,1.0,1.0
24029,3.0,1.0,1.0,0,1,0,0,0,0,1,...,0.0,1.0,3.0,1.0,4.0,4.0,2.0,2.0,2.0,0.0
5882,3.0,2.0,0.0,0,0,0,0,0,0,1,...,0.0,1.0,5.0,2.0,2.0,5.0,2.0,2.0,1.0,0.0
26065,3.0,1.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,4.0,2.0,5.0,4.0,2.0,1.0,1.0
6106,3.0,1.0,1.0,1,0,0,0,0,0,1,...,0.0,1.0,4.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17729,2.0,1.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,2.0,1.0,5.0,4.0,1.0,0.0,0.0
581,3.0,1.0,0.0,0,0,0,0,0,0,1,...,0.0,1.0,4.0,4.0,4.0,5.0,4.0,2.0,0.0,0.0
2169,3.0,2.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,1.0,1.0,5.0,2.0,1.0,1.0,0.0
15598,3.0,1.0,1.0,0,1,0,0,0,0,1,...,0.0,1.0,3.0,2.0,4.0,3.0,2.0,2.0,2.0,2.0


In [55]:
model = DecisionTreeClassifier()
model.fit(xtrain,ytrain)

DecisionTreeClassifier()

In [56]:
pred = model.predict(xtest)

In [57]:
print(classification_report(pred,ytest))
print(confusion_matrix(pred,ytest))

              precision    recall  f1-score   support

           0       0.82      0.85      0.83      6112
           1       0.44      0.39      0.41      1901

    accuracy                           0.74      8013
   macro avg       0.63      0.62      0.62      8013
weighted avg       0.73      0.74      0.73      8013

[[5172  940]
 [1164  737]]


In [58]:
y.value_counts(normalize=True)

0    0.787546
1    0.212454
Name: h1n1_vaccine, dtype: float64

### KNN Algorithm

In [59]:
y = ml_data["h1n1_vaccine"]
x = ml_data.drop("h1n1_vaccine",axis=1)

In [60]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=16)

In [61]:
xtrain

,qualification,income_level,housing_status,age_bracket_35 - 44 Years,age_bracket_45 - 54 Years,age_bracket_55 - 64 Years,age_bracket_65+ Years,race_Hispanic,race_Other or Multiple,race_White,...,is_health_worker,has_health_insur,is_h1n1_vacc_effective,is_h1n1_risky,sick_from_h1n1_vacc,is_seas_vacc_effective,is_seas_risky,sick_from_seas_vacc,no_of_adults,no_of_children
15255,2.0,0.0,1.0,0,0,0,0,1,0,0,...,0.0,1.0,3.0,2.0,4.0,4.0,4.0,2.0,1.0,1.0
24029,3.0,1.0,1.0,0,1,0,0,0,0,1,...,0.0,1.0,3.0,1.0,4.0,4.0,2.0,2.0,2.0,0.0
5882,3.0,2.0,0.0,0,0,0,0,0,0,1,...,0.0,1.0,5.0,2.0,2.0,5.0,2.0,2.0,1.0,0.0
26065,3.0,1.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,4.0,2.0,5.0,4.0,2.0,1.0,1.0
6106,3.0,1.0,1.0,1,0,0,0,0,0,1,...,0.0,1.0,4.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17729,2.0,1.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,2.0,1.0,5.0,4.0,1.0,0.0,0.0
581,3.0,1.0,0.0,0,0,0,0,0,0,1,...,0.0,1.0,4.0,4.0,4.0,5.0,4.0,2.0,0.0,0.0
2169,3.0,2.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,1.0,1.0,5.0,2.0,1.0,1.0,0.0
15598,3.0,1.0,1.0,0,1,0,0,0,0,1,...,0.0,1.0,3.0,2.0,4.0,3.0,2.0,2.0,2.0,2.0


In [62]:
model = KNeighborsClassifier()
model.fit(xtrain,ytrain)

KNeighborsClassifier()

In [63]:
pred = model.predict(xtest)

In [64]:
print(classification_report(pred,ytest))
print(confusion_matrix(pred,ytest))

              precision    recall  f1-score   support

           0       0.94      0.79      0.86      7505
           1       0.08      0.26      0.12       508

    accuracy                           0.76      8013
   macro avg       0.51      0.52      0.49      8013
weighted avg       0.89      0.76      0.81      8013

[[5958 1547]
 [ 378  130]]


In [65]:
y.value_counts(normalize=True)

0    0.787546
1    0.212454
Name: h1n1_vaccine, dtype: float64

### Random Forest Algorithm

In [66]:
y = ml_data["h1n1_vaccine"]
x = ml_data.drop("h1n1_vaccine",axis=1)

In [67]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=16)

In [68]:
xtrain

,qualification,income_level,housing_status,age_bracket_35 - 44 Years,age_bracket_45 - 54 Years,age_bracket_55 - 64 Years,age_bracket_65+ Years,race_Hispanic,race_Other or Multiple,race_White,...,is_health_worker,has_health_insur,is_h1n1_vacc_effective,is_h1n1_risky,sick_from_h1n1_vacc,is_seas_vacc_effective,is_seas_risky,sick_from_seas_vacc,no_of_adults,no_of_children
15255,2.0,0.0,1.0,0,0,0,0,1,0,0,...,0.0,1.0,3.0,2.0,4.0,4.0,4.0,2.0,1.0,1.0
24029,3.0,1.0,1.0,0,1,0,0,0,0,1,...,0.0,1.0,3.0,1.0,4.0,4.0,2.0,2.0,2.0,0.0
5882,3.0,2.0,0.0,0,0,0,0,0,0,1,...,0.0,1.0,5.0,2.0,2.0,5.0,2.0,2.0,1.0,0.0
26065,3.0,1.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,4.0,2.0,5.0,4.0,2.0,1.0,1.0
6106,3.0,1.0,1.0,1,0,0,0,0,0,1,...,0.0,1.0,4.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17729,2.0,1.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,2.0,1.0,5.0,4.0,1.0,0.0,0.0
581,3.0,1.0,0.0,0,0,0,0,0,0,1,...,0.0,1.0,4.0,4.0,4.0,5.0,4.0,2.0,0.0,0.0
2169,3.0,2.0,1.0,0,0,0,1,0,0,1,...,0.0,1.0,5.0,1.0,1.0,5.0,2.0,1.0,1.0,0.0
15598,3.0,1.0,1.0,0,1,0,0,0,0,1,...,0.0,1.0,3.0,2.0,4.0,3.0,2.0,2.0,2.0,2.0


In [70]:
model = RandomForestRegressor()
model.fit(xtrain,ytrain)

RandomForestRegressor()

In [71]:
pred = model.predict(xtest)

In [80]:
print('Mean Absolute Error:', metrics.mean_absolute_error(ytest,pred))
print('Mean Squared Error:', metrics.mean_squared_error(ytest,pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(ytest,pred)))

Mean Absolute Error: 0.2536553101210533
Mean Squared Error: 0.12377334331710968
Root Mean Squared Error: 0.3518143591684536


In [81]:
y.value_counts(normalize=True)

0    0.787546
1    0.212454
Name: h1n1_vaccine, dtype: float64